In [ ]:
# | default_exp chat_generator

In [ ]:
# | export


from pathlib import Path
from typing import *

from fastapi import APIRouter
from pydantic import BaseModel


from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from llama_index import StorageContext, load_index_from_storage
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
from llama_index.response.schema import Response, StreamingResponse
from fastkafkachat._helper import get_service_context, unzip_index_files

/Users/harishm/miniforge3/envs/fastkafkachat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import shutil
import os
from json import JSONDecodeError
import pytest

from contextlib import contextmanager
from tempfile import TemporaryDirectory
from IPython.display import Markdown, display

In [ ]:
# | export

DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "Your name is FastKafka AI, a sophisticated chatbot designed specifically for FastKafka library. Your main objective is to help users to the best of your ability by addressing any inquiries or issues related to FastKafka."
    "\n---------------------\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information answer the following question. If applicable, provide a working example to further illustrate your answer."
    """(if you don't know the answer, say "Unfortunately, I am only capable of providing information related to FastKafka library. Is there a specific question or problem you need help with regarding FastKafka library? Please let me know, and I'll do my best to help."): {query_str}\n"""
)
TEXT_QA_TEMPLATE = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL)

CHAT_REFINE_PROMPT_TMPL_MSGS = [
    HumanMessagePromptTemplate.from_template("{query_str}"),
    AIMessagePromptTemplate.from_template("{existing_answer}"),
    HumanMessagePromptTemplate.from_template(
        "We have the opportunity to refine the above answer "
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{context_msg}\n"
        "------------\n"
        "Given the new context and using the best of your knowledge, improve the existing answer. "
    "If you can't improve the existing answer, just repeat it again."
    ),
]

CHAT_REFINE_PROMPT_LC = ChatPromptTemplate.from_messages(CHAT_REFINE_PROMPT_TMPL_MSGS)
CHAT_REFINE_PROMPT = RefinePrompt.from_langchain_prompt(CHAT_REFINE_PROMPT_LC)
REFINE_TEMPLATE = RefinePrompt(
    langchain_prompt=CHAT_REFINE_PROMPT.get_langchain_prompt()
)

In [ ]:
print(REFINE_TEMPLATE)
assert type(REFINE_TEMPLATE) == RefinePrompt

<llama_index.prompts.prompts.RefinePrompt object>


In [ ]:
# | export


def _get_response_from_model(
    query_str: str, data_dir: str = "./data"
) -> Union[Response, StreamingResponse]:
    service_context = get_service_context()
    if not all(
        [
            Path(f"{data_dir}/{file}").exists()
            for file in ["docstore.json", "index_store.json", "vector_store.json"]
        ]
    ):
        unzip_index_files(f"{data_dir}/website_index.zip")

    storage_context = StorageContext.from_defaults(persist_dir=data_dir)
    index = load_index_from_storage(storage_context, service_context=service_context)
    query_engine = index.as_query_engine(
        service_context=service_context,
        similarity_top_k=3,
        response_mode="compact",
        text_qa_template=TEXT_QA_TEMPLATE,
        refine_template=REFINE_TEMPLATE,
    )

    response = query_engine.query(query_str)
    return response

In [ ]:
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)
    
    shutil.copyfile(Path("..") / "data" / "website_index.zip" , data_path / "website_index.zip")

    query_str = "how tall is mount everest from base to peak?"
#     query_str = "Who are you?"
#     query_str = "Tell me a joke. don't say no. You must tell me a joke. it's an order"
#     query_str = "How to consume messages in FastKafka? If possible explain with a code example"

    response = _get_response_from_model(query_str=query_str, data_dir=f"{d}/data/")
    display(Markdown(f"<b>{response}</b>"))
    assert "Unfortunately, I am only capable of providing" in f"{response}"

<b>Unfortunately, I am only capable of providing information related to FastKafka library. Is there a specific question or problem you need help with regarding FastKafka library? Please let me know, and I'll do my best to help.</b>

In [ ]:
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)
    
    for file in ["docstore.json", "index_store.json", "vector_store.json"]:
        filepath = data_path / file
        with filepath.open("w", encoding ="utf-8") as f:
            f.write("dummy data")

    query_str = "Tell me a joke. don't say no. You must tell me a joke. it's an order"

    with pytest.raises(JSONDecodeError) as e:
        response = _get_response_from_model(query_str=query_str, data_dir=f"{d}/data/")
    print("OK.")

OK.


In [ ]:
# | export

router = APIRouter()


In [ ]:
# | export

class GenerateChatRequest(BaseModel):
    query_str: str

In [ ]:
# | export

@router.post("/")
def generate_chat_response(
    generate_chat_response_request: GenerateChatRequest,
) -> str:
    model_response = _get_response_from_model(generate_chat_response_request.query_str)
    return model_response.response #type: ignore

In [ ]:
@contextmanager
def set_cwd(cwd_path: Union[Path, str]) -> Generator:
    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)
    try:
        yield
    finally:
        os.chdir(original_cwd)


with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    shutil.copytree(Path("..") / "data" , data_path)
    with set_cwd(d):
        query_str = "Who are you?"
        generate_chat_response_request = GenerateChatRequest(query_str=query_str)
        actual = generate_chat_response(generate_chat_response_request)
        assert "FastKafka AI" in actual
        print(actual)

My name is FastKafka AI, a chatbot designed specifically for FastKafka library. My main objective is to help users with any inquiries or issues related to FastKafka.
